In [1]:
from langchain.llms import GPT4All

In [2]:
model = GPT4All(
            model=r"C:\Users\deshi\Code\gpt4all-models\orca-mini-3b-gguf2-q4_0.gguf",
            n_threads=8
        )

In [5]:
with open(r"C:\Users\deshi\Code\whats-up-doc\assets\test_prompt.txt", "r") as file:
    prompt = file.read()

prompt

'Here\'s my function in Python: \n\ndef get_llama_response(prompt: str) -> None:\n    \'\'\'\n    Generate a response from the Llama model.\n\n    Parameters:\n        prompt (str): The user\'s input/question for the model.\n\n    Returns:\n        None: Prints the model\'s response.\n    \'\'\'\n    sequences = llama_pipeline(\n        prompt,\n        do_sample=True,\n        top_k=10,\n        num_return_sequences=1,\n        eos_token_id=tokenizer.eos_token_id,\n        max_length=256,\n    )\n    print("Chatbot:", sequences[0][\'generated_text\'])\n\nGiven the definition of a function in any programming language (particularly Python and C++), please generate it\'s stand-alone documentation. I want it complete with fields like function name, function arguments and return values as well as a detailed explanation of how the function logic works line-by-line. Make it concise and informative to put the documentation into a project documentation file.'

In [4]:
response = model(prompt)

In [6]:
response

''